# Coursera recomender System

## How to Run
* Intall all librearies
* Run all cells
* Go to the last cell for see the recommendation

## Imports

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1.Gathering Data from Coursera via request API

In [3]:
import pandas as pd
import requests

def get_courses_data():
    api_url = "https://api.coursera.org/api/courses.v1"
    params = {
        "start": 0,
        "limit": 10000,
        "fields": "id,name,slug,startDate,primaryLanguages,workload,isPartOfSpecialization"
    }
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        courses = data["elements"]
        
        course_data = []
        for course in courses:
            course_id = course["id"]
            course_name = course["name"]
            course_slug = course["slug"]
            start_date = course.get("startDate", "")
            end_date = course.get("endDate", "")
            primary_languages = course.get("primaryLanguages", [])
            workload = course.get("workload", "")
            primary_category = course.get("primaryCategory", "")
            instructor_ids = course.get("instructorIds", [])
            is_part_of_specialization = course.get("isPartOfSpecialization", False)
            skills = course.get("skills", [])
            audience = course.get("audience", "")
            course_format = course.get("courseFormat", "")
            
            course_data.append({
                "ID": course_id,
                "Name": course_name,
                "Slug": course_slug,
                "Start Date": start_date,
                "Primary Languages": primary_languages[0].split('-')[0],
                "Workload": workload,
                #"primary_category":primary_category,
                #"Instructor IDs": instructor_ids,
                "Is Part of Specialization": is_part_of_specialization,
            })
        
        df = pd.DataFrame(course_data)
        return df
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
        return None

coursesdf = get_courses_data()

In [4]:
coursesdf['Start Date'] = pd.to_datetime(coursesdf['Start Date'])
coursesdf['Start Date'] = coursesdf['Start Date'].dt.strftime('%d-%m-%Y %H:%M')
coursesdf['Start Date'].describe()

count                10000
unique                   7
top       01-01-1970 00:27
freq                  4126
Name: Start Date, dtype: object

In [5]:
"""
Do not run this cell, it has already done.
The csv file has been generated.
"""

# coursesdf.to_csv('Coursera.csv', index=False)

'\nDo not run this cell, it has already done.\nThe csv file has been generated.\n'

In [6]:
courses = pd.read_csv("Coursera.csv")
courses

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...
...,...,...,...,...,...,...,...
3517,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,Beginner,4.6,https://www.coursera.org/learn/python-data-vis...,"In the capstone, students will build a series ...",Databases syntax analysis web Data Visuali...
3518,Patrick Henry: Forgotten Founder,University of Virginia,Intermediate,4.9,https://www.coursera.org/learn/henry,"�Give me liberty, or give me death:� Rememberi...",retirement Causality career history of the ...
3519,Business intelligence and data analytics: Gene...,Macquarie University,Advanced,4.6,https://www.coursera.org/learn/business-intell...,�Megatrends� heavily influence today�s organis...,analytics tableau software Business Intellig...
3520,Rigid Body Dynamics,Korea Advanced Institute of Science and Techno...,Beginner,4.6,https://www.coursera.org/learn/rigid-body-dyna...,"This course teaches dynamics, one of the basic...",Angular Mechanical Design fluid mechanics F...


## 2.Database

### 2-1.Save to DB

In [8]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

def save_df_to_mysql(df, database, table_name, host='localhost', user='your_username', password='your_password'):
    # Create a MySQL connection
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password
    )
    
    # Create the database if it doesn't exist
    cursor = connection.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
    cursor.close()
    
    # Create SQLAlchemy engine
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    
    # Save the DataFrame to the MySQL table
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    
    # Close the connection
    connection.close()
    print(f"DataFrame saved to the '{database}.{table_name}' table.")

In [9]:
database_name = "DBLAB_final_project"
table_name = "test3"
save_df_to_mysql(courses, database_name, table_name, host='localhost', user='root', password='pdne')

DataFrame saved to the 'DBLAB_final_project.test3' table.


### 2-2.Connect to DB

In [1]:
import mysql
import mysql.connector
# run this every time
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="pdne",
 
)
mycursor = mydb.cursor()

mycursor.execute("SHOW DATABASES")
for db in mycursor:
    print(db)

('Coursera',)
('Coursera_DBLAB',)
('DB_Lab',)
('DBLAB_final_project',)
('information_schema',)
('myDatabase',)
('mysql',)
('performance_schema',)
('Session_Data',)
('Study',)
('sys',)
('TopLearn_Project',)


In [5]:
import pandas as pd
mycursor.execute("USE DBLAB_final_project")
data = pd.read_sql('SELECT * FROM test3', mydb)
data

/var/folders/bf/9616ns414yg2qn2p8czl2sr00000gn/T/ipykernel_18149/967033774.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  courses_df = pd.read_sql('SELECT * FROM test3', mydb)


,ID,Name,Slug,Start Date,Primary Languages,Workload,Is Part of Specialization
0,NJSdGN71Eeq4CApSN3OTvQ,Make Your Pick-Ups Look Cool in Unity (Intro t...,make-pick-ups-look-cool-unity-introduction-ani...,1598035773172,en,2 hours,0
1,DMkcgX7LEeyRTg6FtAvfBw,Intégrer des applications dans votre Dashboard...,integrer-applications-dashboard-hootsuite,1644614698077,fr,2 heures,0
2,YLO0oGSUEeyIUg4Qv2RsBQ,كيفية استعمال التطبيق هووتسويت,getting-started-with-hootsuite-ar,1642193091376,ar,2 hours,0
3,69Bku0KoEeWZtA4u62x6lQ,Gamification,gamification,1447095621493,en,4-8 hours/week,0
4,0HiU7Oe4EeWTAQ4yevf_oQ,Dealing With Missing Data,missing-data,1471901099812,en,"4 weeks of study, 1-2 hours/week",0
...,...,...,...,...,...,...,...
95,XXZBGc97EeufchLeGgZGZQ,Relational Database Administration (DBA),relational-database-administration,1637617419532,en,"5 weeks of study, 2-4 hours / week",0
96,M1lJ0BrjEeui5Apx-DHBkQ,Waits in Selenium Test Automation Tool,waits-in-selenium-test-automation-tool,1604094399504,en,2 hours,0
97,LZZg6vhQEeWfYgqbi1xsdw,Chinese Characters for beginner 汉字,hanzi,1477679733728,zh,5-7 hours/week,0
98,wFc3FhwIEeyf4Ar-aygVCw,Introducción a Hootsuite,introduccion-a-hootsuite,1633635891410,es,2 hours,0


## 3.Dataframe and Analysis

In [14]:
data.head(5)

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [15]:
data.shape

(3522, 7)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


In [17]:
data.isnull().sum()

Course Name           0
University            0
Difficulty Level      0
Course Rating         0
Course URL            0
Course Description    0
Skills                0
dtype: int64

In [18]:
data['Difficulty Level'].value_counts()

Beginner          1444
Advanced          1005
Intermediate       837
Conversant         186
Not Calibrated      50
Name: Difficulty Level, dtype: int64

In [19]:
data['Course Rating'].value_counts()

4.7               740
4.6               623
4.8               598
4.5               389
4.4               242
4.9               180
4.3               165
4.2               121
5                  90
4.1                85
Not Calibrated     82
4                  51
3.8                24
3.9                20
3.6                18
3.7                18
3.5                17
3.4                13
3                  12
3.2                 9
3.3                 6
2.9                 6
2.6                 2
2.8                 2
2.4                 2
1                   2
2                   1
3.1                 1
2.5                 1
1.9                 1
2.3                 1
Name: Course Rating, dtype: int64

In [20]:
data['University'].value_counts()

Coursera Project Network                      562
University of Illinois at Urbana-Champaign    138
Johns Hopkins University                      110
University of Michigan                        101
University of Colorado Boulder                101
                                             ... 
GitLab                                          1
Yeshiva University                              1
University of Glasgow                           1
Laureate Education                              1
The World Bank Group                            1
Name: University, Length: 184, dtype: int64

In [21]:
data['Skills'].value_counts()

Google Cloud Platform  Big Data  Cloud Infrastructure  google storage  Cloud Storage  bigquery  Cloud Platforms  Kubernetes  Cloud Computing  Google App Engine information-technology cloud-computing                                  8
art  listening  emotions  Sound Design  music  signature  Audio Production  Writing  microphone  sound arts-and-humanities music-and-art                                                                                                4
Python Programming  python libraries  analysis  semantics  Computer Programming  syntax  Data Analysis  Pandas  python syntax and semantics  Numpy data-science data-analysis                                                           4
analysis  Machine Learning  Data Mining  Artificial Neural Networks  Regression Analysis  Data Analysis  data reporting  Human Learning  Regression  Big Data data-science data-analysis                                                4
Machine Learning  artificial general intelligence  application d

In [22]:
data['Course Name']

0       Write A Feature Length Screenplay For Film Or ...
1       Business Strategy: Business Model Canvas Analy...
2                           Silicon Thin Film Solar Cells
3                                    Finance for Managers
4            Retrieve Data using Single-Table SQL Queries
                              ...                        
3517    Capstone: Retrieving, Processing, and Visualiz...
3518                     Patrick Henry: Forgotten Founder
3519    Business intelligence and data analytics: Gene...
3520                                  Rigid Body Dynamics
3521    Architecting with Google Kubernetes Engine: Pr...
Name: Course Name, Length: 3522, dtype: object

## Required Columns for System

#### Important columns to be used in recommendation system : 

- Course Ratings : Users may want to see courses based on rates
- Skills : Users may want to see courses based on same skills
- Difficulty Level : Similar courses as per difficulty level 

#### Columns not used for the recommendation system :

- Course Name : Names of the courses
- Course Description : Similar courses may have similar course description but not useful for us
- University : Same university might offer multiple courses in different domains which the user might not want to see
- Course URL : No significance in the recommendation system

In [23]:
data = data[['Course Name','Difficulty Level','Course Rating','Skills']]
data.head(5)

,Course Name,Difficulty Level,Course Rating,Skills
0,Write A Feature Length Screenplay For Film Or ...,Beginner,4.8,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Beginner,4.8,Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,Advanced,4.1,chemistry physics Solar Energy film lambda...
3,Finance for Managers,Intermediate,4.8,accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Beginner,4.6,Data Analysis select (sql) database manageme...


## Data Pre-Processing

An important part of the process is to pre-process the data into usable format for the recommendation system

In [24]:
# Removing spaces between the words (Lambda funtions can be used as well)

data['Course Name'] = data['Course Name'].str.replace(' ',',')
data['Course Name'] = data['Course Name'].str.replace(',,',',')
data['Course Name'] = data['Course Name'].str.replace(':','')

#removing paranthesis from skills columns 
data['Skills'] = data['Skills'].str.replace('(','')
data['Skills'] = data['Skills'].str.replace(')','')

data.head(5)

C:\Users\USER\AppData\Local\Temp\ipykernel_12160\1865971621.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Skills'] = data['Skills'].str.replace('(','')
C:\Users\USER\AppData\Local\Temp\ipykernel_12160\1865971621.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Skills'] = data['Skills'].str.replace(')','')


,Course Name,Difficulty Level,Course Rating,Skills
0,"Write,A,Feature,Length,Screenplay,For,Film,Or,...",Beginner,4.8,Drama Comedy peering screenwriting film D...
1,"Business,Strategy,Business,Model,Canvas,Analys...",Beginner,4.8,Finance business plan persona user experienc...
2,"Silicon,Thin,Film,Solar,Cells",Advanced,4.1,chemistry physics Solar Energy film lambda...
3,"Finance,for,Managers",Intermediate,4.8,accounts receivable dupont analysis analysis...
4,"Retrieve,Data,using,Single-Table,SQL,Queries",Beginner,4.6,Data Analysis select sql database management...


In [25]:
copy_df = data.copy()
copy_df.head(5)

,Course Name,Difficulty Level,Course Rating,Skills
0,"Write,A,Feature,Length,Screenplay,For,Film,Or,...",Beginner,4.8,Drama Comedy peering screenwriting film D...
1,"Business,Strategy,Business,Model,Canvas,Analys...",Beginner,4.8,Finance business plan persona user experienc...
2,"Silicon,Thin,Film,Solar,Cells",Advanced,4.1,chemistry physics Solar Energy film lambda...
3,"Finance,for,Managers",Intermediate,4.8,accounts receivable dupont analysis analysis...
4,"Retrieve,Data,using,Single-Table,SQL,Queries",Beginner,4.6,Data Analysis select sql database management...


In [26]:
copy_df['Course Rating'].iloc[1]

'4.8'

## Yes/No questions functions

### Rating
- User see the "Should the rate of the course be less than 3.5? (yes/no): " question
- User will click on yes/no button at tkinter envierment
- We check that the user clicked the yes button or no button
- yes: Less than 3.5
- no: Greater or equal than 3.5

In [27]:
def filter_courses_by_rating(df, userInput):
    if userInput:
        filtered_df = df[df['Course Rating'] < '3.5']
    else:
        filtered_df = df[df['Course Rating'] >= '3.5']
    return filtered_df

In [28]:
# # Test the above cell (yes)
# user_input = input("Should the rate of the course be less than 3.5? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_rating(copy_df, userInput)
# filtered_courses

In [29]:
# # Test the above cell (no)
# user_input = input("Should the rate of the course be less than 3.5? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_rating(copy_df, userInput)
# filtered_courses

## Dificulty

### Beginer
- User see the "Should the Difficulty of the course be Beginer? (yes/no): " question
- User will click on yes/no button at tkinter envierment
- We check that the user clicked the yes button or no button
- yes: Beginer Difficulty
- no: return

In [30]:
def filter_courses_by_difficulty_beginner(df, userInput):
    if userInput:
        filtered_df = df[df['Difficulty Level'] == 'Beginner']
    else:
        return df
    return filtered_df

In [31]:
# # Test the above cell (yes)
# user_input = input("Should the Difficulty of the course be Beginer? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_difficulty_beginner(copy_df, userInput)
# filtered_courses

In [32]:
# # Test the above cell (no)
# user_input = input("Should the Difficulty of the course be Beginer? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_difficulty_beginner(copy_df, userInput)
# filtered_courses

### Intermediate
- User see the "Should the Difficulty of the course be Intermediate? (yes/no): " question
- User will click on yes/no button at tkinter envierment
- We check that the user clicked the yes button or no button
- yes: Beginer Difficulty
- no: return

In [33]:
def filter_courses_by_difficulty_intermediate(df, userInput):
    if userInput:
        filtered_df = df[df['Difficulty Level'] == 'Intermediate']
    else:
        return df
    return filtered_df

In [34]:
# # Test the above cell (yes)
# user_input = input("Should the Difficulty of the course be Intermediate? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_difficulty_intermediate(copy_df, userInput)
# filtered_courses

In [35]:
# # Test the above cell (no)
# user_input = input("Should the Difficulty of the course be Intermediate? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_difficulty_intermediate(copy_df, userInput)
# filtered_courses

### Advanced
- User see the "Should the Difficulty of the course be Advanced? (yes/no): " question
- User will click on yes/no button at tkinter envierment
- We check that the user clicked the yes button or no button
- yes: Beginer Difficulty
- no: return

In [36]:
def filter_courses_by_difficulty_advanced(df, userInput):
    if userInput:
        filtered_df = df[df['Difficulty Level'] == 'Advanced']
    else:
        return df
    return filtered_df

In [37]:
# # Test the above cell (yes)
# user_input = input("Should the Difficulty of the course be Advanced? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_difficulty_advanced(copy_df, userInput)
# filtered_courses

In [38]:
# # Test the above cell (no)
# user_input = input("Should the Difficulty of the course be Advanced? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_difficulty(copy_df, userInput)
# filtered_courses

## Skills

### Data Analysis
- User see the "Should the Skills of the course contain Data Analysis? (yes/no): " question
- User will click on yes/no button at tkinter envierment
- We check that the user clicked the yes button or no button
- yes: Beginer Difficulty
- no: return

In [39]:
def filter_courses_by_skills_data_analysis(df, userInput):
    if userInput:
        filtered_df = df[df['Skills'].str.contains('Data Analysis', case=False, na=False)]
    else:
        return df
    return filtered_df

In [40]:
# # Test the above cell (yes)
# user_input = input("Should the Skills of the course contain Data Analysis? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_skills_data_analysis(copy_df, userInput)
# filtered_courses

In [41]:
# # Test the above cell (no)
# user_input = input("Should the Skills of the course contain Data Analysis? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_skills_data_analysis(copy_df, userInput)
# filtered_courses

### Programming
- User see the "Should the Skills of the course contain Programming? (yes/no): " question
- User will click on yes/no button at tkinter envierment
- We check that the user clicked the yes button or no button
- yes: Beginer Difficulty
- no: return

In [42]:
def filter_courses_by_skills_programming(df, userInput):
    if userInput:
        filtered_df = df[df['Skills'].str.contains('Programming', case=False, na=False)]
    else:
        return df
    return filtered_df

In [43]:
# # Test the above cell (yes)
# user_input = input("Should the Skills of the course contain Programming? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_skills_programming(copy_df, userInput)
# filtered_courses

In [44]:
# # Test the above cell (no)
# user_input = input("Should the Skills of the course contain Programming? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_skills_programming(copy_df, userInput)
# filtered_courses

### Art
- User see the "Should the Skills of the course contain Art? (yes/no): " question
- User will click on yes/no button at tkinter envierment
- We check that the user clicked the yes button or no button
- yes: Beginer Difficulty
- no: return

In [45]:
def filter_courses_by_skills_art(df, userInput):
    if userInput:
        filtered_df = df[df['Skills'].str.contains('Art', case=False, na=False)]
    else:
        return df
    return filtered_df

In [46]:
# # Test the above cell (yes)
# user_input = input("Should the Skills of the course contain Art? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_skills_art(copy_df, userInput)
# filtered_courses

In [47]:
# # Test the above cell (no)
# user_input = input("Should the Skills of the course contain Art? (yes/no): ")
# if user_input == 'yes':
#     userInput = 1.0
# else:
#     userInput = 0.0

# filtered_courses = filter_courses_by_skills_art(copy_df, userInput)
# filtered_courses

## 4.GUI environment

In [48]:
import tkinter as tk
#import pandastable as pdtk

class CourseRecommendationApp(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Course Recommendation")
        self.geometry("300x200")

        self.answers = []
        self.filtered_courses = copy_df

        self.create_welcome_page()

    def create_welcome_page(self):
        self.welcome_frame = tk.Frame(self)
        self.welcome_frame.pack(fill=tk.BOTH, expand=True)

        welcome_label = tk.Label(self.welcome_frame, text="Welcome to Course Recommendation")
        welcome_label.pack(pady=20)

        start_button = tk.Button(self.welcome_frame, text="Start", command=self.start_questions)
        start_button.pack()

    def start_questions(self):
        self.welcome_frame.pack_forget()
        self.questions = [
            "Should the rate of the course be less than 3.5? (yes/no): ",
            "Should the Difficulty of the course be Beginer? (yes/no): ",
            "Should the Difficulty of the course be Advanced? (yes/no): ",
            "Should the Difficulty of the course be Intermediate? (yes/no): ",
            "Should the Skills of the course contain Data Analysis? (yes/no): ",
            "Should the Skills of the course contain Programming? (yes/no): ",
            "Should the Skills of the course contain Art? (yes/no): ",
        ]
        self.current_question = 0
        self.create_question_page()

    def create_question_page(self):
        self.question_frame = tk.Frame(self)
        self.question_frame.pack(fill=tk.BOTH, expand=True)

        question_label = tk.Label(self.question_frame, text=self.questions[self.current_question])
        question_label.pack(pady=20)

        yes_button = tk.Button(self.question_frame, text="Yes", command=lambda: self.answer_question("Yes"))
        yes_button.pack(side=tk.LEFT, padx=20)

        no_button = tk.Button(self.question_frame, text="No", command=lambda: self.answer_question("No"))
        no_button.pack(side=tk.RIGHT, padx=20)


    def answer_question(self, answer):
        self.answers.append(answer)
        self.current_question += 1
            
        if self.current_question < len(self.questions):
            self.question_frame.pack_forget()
            self.create_question_page()
        else:
            self.show_results()

    def show_results(self):
        self.question_frame.pack_forget()

        results_frame = tk.Frame(self)
        results_frame.pack(fill=tk.BOTH, expand=True)

        results_label = tk.Label(results_frame, text="You have completed the questions.")
        results_label.pack(pady=20)

        answers_label = tk.Label(results_frame, text=f"Your answers: {self.answers}")
        answers_label.pack()

        end_button = tk.Button(results_frame, text="End", command=self.show_dataframe)
        end_button.pack(pady=10)
        
    def show_dataframe(self):
        # # Create a new window to display the DataFrame
        # df_window = tk.Toplevel(self)
        # df_window.title("DataFrame")

        # # Create a DataFrame widget and add it to the window
        # df_widget = tk.Frame(df_window)
        # df_widget.pack(fill=tk.BOTH, expand=True)

        # # Create a scrollbar for the DataFrame widget
        # scrollbar = tk.Scrollbar(df_widget)
        # scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # # Create a Canvas for the DataFrame widget
        # canvas = tk.Canvas(df_widget, yscrollcommand=scrollbar.set)
        # canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # # Add the DataFrame to the Canvas
        # df_table = pdtk.Table(canvas, dataframe=filtered_courses)
        # df_table.show()

        # # Configure the scrollbar to work with the Canvas
        # scrollbar.config(command=canvas.yview)
        # canvas.config(scrollregion=canvas.bbox(tk.ALL))
        
        
        self.destroy()


app = CourseRecommendationApp()
app.mainloop()

In [49]:
print(app.answers)

['Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No']


In [50]:
user = [0.0] * len(app.answers)
for i in range(len(app.answers)):
    if app.answers[i] == 'Yes':
        user[i] = 1.0
    else:
        user[i] = 0.0
        
print(user)

[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]


In [51]:
rating = filter_courses_by_rating(copy_df, user[0])
 
beginner = filter_courses_by_difficulty_beginner(rating, user[1])

advanced = filter_courses_by_difficulty_advanced(beginner, user[2])

intermediate = filter_courses_by_difficulty_intermediate(advanced, user[3])

analysis = filter_courses_by_skills_data_analysis(intermediate, user[4])

programming = filter_courses_by_skills_programming(analysis, user[5])

filtered_courses = filter_courses_by_skills_art(programming, user[6])

filtered_courses

,Course Name,Difficulty Level,Course Rating,Skills
74,"Predictive,Modeling,and,Analytics",Advanced,3,Logistic Regression analytics predictive ana...
